In [ ]:
! pip install PyMuPDF

In [13]:

import fitz
c="data0"
a= "generatedWehi"
b= "18465975MDXN[20]"
fname = a
# Open the PDF file
doc = fitz.open(fname+".pdf")

# Iterate through the pages of the PDF and extract the text
out = open(fname + ".txt", "wb")
for page in doc:
    text = page.get_text().encode("utf8")
    out.write(text)

doc.close()
out.close()

In [14]:
#change the regex pattern so it can match the new format
import re

def extract_patient_info(text):
    patient_info = {}
    
    keyword1 = ["Name","Lab No","Ext Ref","Collected","Received","Ext Ref","Specimen","Requester","Referral Lab"]
    keyword2 = ["URN","DOB","Sex"]
    for w in keyword1:
        match = re.search(f"(?<={w}:\s)[^\n]*\S", text)
        if match:
            patient_info[w] = match.group().strip()
        else:
            patient_info[w] = ""
    for w in keyword2:
        match = re.search(f"{w}:\s+([^\n]+)", text)
        if match:
            patient_info[w] = match.group(1).strip()
        else:
            patient_info[w] = ""       
#     for w in keyword2:
#         match = re.search(f"{w}\s+([^\n]+)", text).group(1).strip()
#         if match:
#             patient_info[w] = match
#         else:
#             patient_info[w] = ""
    return patient_info

with open(fname + ".txt", "r", encoding='utf-8') as f:
    text = f.read()
    patient_info = extract_patient_info(text)
    print(patient_info)

{'Name': 'Nell Cox', 'Lab No': '17999458', 'Ext Ref': '1745766441', 'Collected': '04-Jun-2022', 'Received': '04-Oct-2023', 'Specimen': 'Muscle biopsy test', 'Requester': 'DrJoan Lewis', 'Referral Lab': 'Dorevitch Pathology', 'URN': 'PMEX347746', 'DOB': '04-Dec-1994', 'Sex': 'M'}


In [15]:
#remove the extra "," when extracting "VARIANT" field
def extract_variants(text):
    num =0 #number of variants recorded in teh report
    result = []
    #remove irrelecant sections and the (%)
    match = re.search(r"(ASSUMED ORIGIN.*VRF – variant read frequency)", text, flags=re.DOTALL)
    preprocess = match.group(1)
    preprocess = re.sub(r"\n\(%\)", "", preprocess)
    preprocess = re.sub("VRF – variant read frequency", "", preprocess)
    
    
    # Split the preprocessed text by lines and remove empty lines
    lines = [line.strip() for line in preprocess.split('\n') if line.strip()]    
    
    #split the headers and table contents
    header = lines[:5]
    lines = lines[5:]
    
    #in some circanstances, viriants may be split to two lines. join the two lines
    pattern_VARIANT = r"^(c|p)\.[A-Za-z0-9()>*=]+"
    i = 0
    while (i<len(lines)):
        match = re.match(pattern_VARIANT,lines[i])
        if match:
            temp = lines.pop(i+1)
            lines[i] = lines[i] + temp
        i+=1
        
    pattern_bracket = r".*\([a-zA-Z]+$"
    i = 0
    while i < len(lines):
        match = re.match(pattern_bracket,lines[i])
        if match:
            temp = lines.pop(i+1)
            lines[i] = lines[i] + temp
        i+=1

    #put result into a dictionary
    j = 0
    while (j<len(lines)):
        dic = {}
        dic[header[0]] = lines[j]
        dic[header[1]] = lines[j+1]
        dic[header[2]] = lines[j+2]
        dic[header[3]] = lines[j+3]
        dic[header[4]] = lines[j+4]
        result.append(dic)
        j = j+5
    return result

with open(fname + ".txt", "r", encoding='utf-8') as f:
    text = f.read()
    a = extract_variants(text)
    print(a)

2
[{'ASSUMED ORIGIN': 'Somatic', 'GENE': 'FLT3', 'VARIANT': 'FLT3-ITD (allelicratio 0.76)', 'VRF (%)': '14', 'CLINICAL SIGNIFICANCE IN AML': 'DIAGNOSTIC'}, {'ASSUMED ORIGIN': 'Eukaryotic', 'GENE': 'ALDH2', 'VARIANT': 'c.2842del,p.(Glu948Argfs*3)', 'VRF (%)': '11', 'CLINICAL SIGNIFICANCE IN AML': 'PROGNOSTIC / DRUG TARGET'}, {'ASSUMED ORIGIN': 'Prokaryotic', 'GENE': 'CFTR', 'VARIANT': 'c.2645G>A,p.(Arg882His)', 'VRF (%)': '11', 'CLINICAL SIGNIFICANCE IN AML': 'CLONAL MARKER'}]


In [16]:
import json 
with open(fname + ".txt", "r", encoding='utf-8') as f:
    text = f.read()
    patient_info = extract_patient_info(text)
    reportable_variants = extract_variants(text)
    
    dic = {"patient_info":patient_info, "reportable_variants":reportable_variants }
    result = json.dumps(dic, indent = 3)
    
    jsonFile = open(fname+".json", "w")
    jsonFile.write(result)
    jsonFile.close()

2


In [11]:
#remove the txt
import os
os.remove(fname+".txt")